# Phase 6: Large-Scale LSH Benchmark - Supplemental

**Objective:** Benchmark LSH blocking at full dataset scale (933K+ tier-1 NPI records × 1.175M Medicare),
sweeping hyperparameters and comparing against exact-NPI matching and traditional blocking.

**Input:** Cleaned parquet files from Phase 2 (`../artifacts/phase2_preprocessing/`)

**Output:** `../artifacts/phase6_lsh_benchmark/`
- `lsh_benchmark_results.csv` — hyperparameter sweep results
- `lsh_vs_baselines.csv` — LSH vs exact NPI vs traditional blocking
- `lsh_benchmark_charts.png` — visualization

**Key Questions:**
1. How does LSH runtime scale with `num_perm` and `threshold`?
2. What is the precision/recall tradeoff at different similarity thresholds?
3. Does LSH find matches that exact NPI misses (and vice versa)?
4. What is the optimal configuration for production use?

## 4.0 Setup & Configuration

In [4]:
import os, gc, time, math
import pandas as pd
import numpy as np
from itertools import product

INPUTDIR = "../artifacts/phase2_preprocessing"
OUTPUTDIR = "../artifacts/phase6_lsh_benchmark"
os.makedirs(OUTPUTDIR, exist_ok=True)

# Reproducibility
np.random.seed(42)
print("Setup complete.")


Setup complete.


## 4.1 Load Full-Scale Datasets

In [5]:
print("=" * 60)
print("4.1 LOAD FULL-SCALE DATASETS")
print("=" * 60)

opclean = pd.read_parquet(os.path.join(INPUTDIR, "open_payments_clean.parquet"))
medclean = pd.read_parquet(os.path.join(INPUTDIR, "medicare_clean.parquet"))

# Separate tiers
op_tier1 = opclean[opclean["linkage_tier"] == "tier1_npi"]
op_tier2 = opclean[opclean["linkage_tier"] == "tier2_fuzzy"].copy().reset_index(drop=True)
op_unmatchable = opclean[opclean["linkage_tier"] == "unmatchable"]

print(f"Full OP dataset:      {len(opclean):>12,}")
print(f"  Tier-1 (NPI):       {len(op_tier1):>12,}")
print(f"  Tier-2 (fuzzy):     {len(op_tier2):>12,}")
print(f"  Unmatchable:        {len(op_unmatchable):>12,}")
print(f"Medicare backbone:    {len(medclean):>12,}")
print(f"Full cross-product:   {len(op_tier2) * len(medclean):>15,} pairs")

del opclean; gc.collect()
print("\nDatasets loaded.")


4.1 LOAD FULL-SCALE DATASETS
Full OP dataset:           969,703
  Tier-1 (NPI):            933,615
  Tier-2 (fuzzy):            4,683
  Unmatchable:              31,405
Medicare backbone:       1,175,281
Full cross-product:     5,503,840,923 pairs

Datasets loaded.


## 4.2 Baseline 1 — Exact NPI Matching

The simplest and fastest approach: inner-join on NPI. This is the gold standard for
tier-1 records and serves as our ground-truth reference for precision.

In [8]:
op_tier1.columns

Index(['Covered_Recipient_NPI', 'Covered_Recipient_Profile_ID',
       'Covered_Recipient_First_Name', 'Covered_Recipient_Last_Name',
       'Recipient_Primary_Business_Street_Address_Line1', 'Recipient_City',
       'Recipient_State', 'Recipient_Zip5', 'NPI_VALID', 'LINKABLE',
       'linkage_tier', 'FIRST_NAME_SOUNDEX', 'LAST_NAME_SOUNDEX',
       'FIRST_NAME_METAPHONE', 'LAST_NAME_METAPHONE', 'total_payment_amount',
       'payment_count', 'avg_payment', 'max_payment', 'unique_manufacturers',
       'min_payment_date', 'max_payment_date'],
      dtype='object')

In [9]:
print("=" * 60)
print("4.2 BASELINE 1: EXACT NPI MATCHING")
print("=" * 60)

t0 = time.time()

# Standardize NPI dtypes
op_npi = pd.to_numeric(op_tier1["Covered_Recipient_NPI"], errors="coerce").dropna().astype("int64")
med_npi = pd.to_numeric(medclean["Rndrng_NPI"], errors="coerce").dropna().astype("int64")

# Set intersection
op_npi_set = set(op_npi.unique())
med_npi_set = set(med_npi.unique())
npi_overlap = op_npi_set & med_npi_set

elapsed_npi = time.time() - t0

print(f"Unique OP tier-1 NPIs:       {len(op_npi_set):>12,}")
print(f"Unique Medicare NPIs:        {len(med_npi_set):>12,}")
print(f"NPI overlap (exact matches): {len(npi_overlap):>12,}")
print(f"Match rate (of OP tier-1):   {len(npi_overlap)/len(op_npi_set):.1%}")
print(f"Runtime:                     {elapsed_npi:.2f}s")


4.2 BASELINE 1: EXACT NPI MATCHING
Unique OP tier-1 NPIs:            933,615
Unique Medicare NPIs:           1,175,281
NPI overlap (exact matches):      542,329
Match rate (of OP tier-1):   58.1%
Runtime:                     0.65s


## 4.3 Baseline 2 — Traditional Blocking (State + Soundex)

Run Strategy B (State + Last Name Soundex) on tier-2 fuzzy records as the
traditional blocking baseline. Already benchmarked in Phase 3 at ~1M pairs.

In [12]:
print("=" * 60)
print("4.3 BASELINE 2: TRADITIONAL BLOCKING (STRATEGY B)")
print("=" * 60)

t0 = time.time()

op_tier2["block_B"] = op_tier2["LAST_NAME_SOUNDEX"].fillna("") + "|" + op_tier2["Recipient_State"].fillna("")
medclean["block_B"] = medclean["LAST_NAME_SOUNDEX"].fillna("") + "|" + medclean["Rndrng_Prvdr_State_Abrvtn"].fillna("")

pairs_B = (
    op_tier2[["block_B"]].reset_index().rename(columns={"index": "index_op"})
    .merge(
        medclean[["block_B"]].reset_index().rename(columns={"index": "index_med"}),
        on="block_B"
    )[["index_op", "index_med"]]
)

elapsed_B = time.time() - t0
fullcross = len(op_tier2) * len(medclean)
rr_B = 1 - len(pairs_B) / fullcross

print(f"Tier-2 records:       {len(op_tier2):>12,}")
print(f"Medicare records:     {len(medclean):>12,}")
print(f"Candidate pairs (B):  {len(pairs_B):>12,}")
print(f"Reduction ratio:      {rr_B*100:.6f}%")
print(f"Runtime:              {elapsed_B:.2f}s")

# Clean up
op_tier2.drop(columns="block_B", inplace=True)
medclean.drop(columns="block_B", inplace=True)
set_B = set(map(tuple, pairs_B[["index_op", "index_med"]].values))

del pairs_B; gc.collect()


4.3 BASELINE 2: TRADITIONAL BLOCKING (STRATEGY B)
Tier-2 records:              4,683
Medicare records:        1,175,281
Candidate pairs (B):       427,752
Reduction ratio:      99.992228%
Runtime:              0.84s


628

## 4.4 LSH Hyperparameter Sweep

Sweep `num_perm` (MinHash signature size) and `threshold` (Jaccard similarity cutoff)
to find the optimal LSH configuration. We use `datasketch` for production-grade LSH.

**Grid:**
- `num_perm`: [64, 128, 256]
- `threshold`: [0.3, 0.4, 0.5, 0.6, 0.7]
- Q-gram size: 3 (trigrams, fixed)

In [14]:
print("=" * 60)
print("4.4 LSH HYPERPARAMETER SWEEP")
print("=" * 60)

try:
    from datasketch import MinHash, MinHashLSH
    print("datasketch ready")
except ImportError:
    !pip install datasketch -q
    from datasketch import MinHash, MinHashLSH
    print("datasketch installed")

Q = 3  # trigram size

def make_minhash(text, num_perm):
    """Create MinHash from text using character q-grams."""
    m = MinHash(num_perm=num_perm)
    if not isinstance(text, str) or len(text) < Q:
        return None
    for i in range(len(text) - Q + 1):
        m.update(text[i:i+Q].encode("utf-8"))
    return m

def name_string(first, last, state):
    """Concatenate name fields for hashing."""
    parts = [str(x).strip().upper() for x in (first, last, state) if pd.notna(x)]
    return " ".join(p for p in parts if p and p != "NAN")

# Precompute name strings (one-time cost)
t0 = time.time()
op_tier2["_ns"] = op_tier2.apply(
    lambda r: name_string(r["Covered_Recipient_First_Name"], 
                          r["Covered_Recipient_Last_Name"], 
                          r["Recipient_State"]), axis=1
)
medclean["_ns"] = medclean.apply(
    lambda r: name_string(r["Rndrng_Prvdr_First_Name"], 
                          r["Rndrng_Prvdr_Last_Org_Name"], 
                          r["Rndrng_Prvdr_State_Abrvtn"]), axis=1
)
print(f"Name strings built in {time.time()-t0:.1f}s")

# Get common states for within-state LSH
states_common = sorted(
    set(op_tier2["Recipient_State"].dropna().unique()) &
    set(medclean["Rndrng_Prvdr_State_Abrvtn"].dropna().unique())
)
print(f"States in common: {len(states_common)}")


4.4 LSH HYPERPARAMETER SWEEP
datasketch ready
Name strings built in 9.5s
States in common: 53


In [16]:
# ---- Sweep ---- limited due to lime contraints, else run on different perms and threshholds
NUM_PERMS = [128]
THRESHOLDS = [0.5]

sweep_results = []

for num_perm, threshold in product(NUM_PERMS, THRESHOLDS):
    print(f"\n--- num_perm={num_perm}, threshold={threshold:.1f} ---")
    t0 = time.time()

    lsh_pairs = set()

    for state in states_common:
        op_st = op_tier2[op_tier2["Recipient_State"] == state]
        med_st = medclean[medclean["Rndrng_Prvdr_State_Abrvtn"] == state]
        if len(op_st) == 0 or len(med_st) == 0:
            continue

        # Build LSH index from Medicare side
        lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
        med_minhashes = {}
        for idx, row in med_st.iterrows():
            mh = make_minhash(row["_ns"], num_perm)
            if mh:
                try:
                    lsh.insert(f"m_{idx}", mh)
                    med_minhashes[idx] = mh
                except ValueError:
                    pass

        # Query OP side
        for idx_op, row_op in op_st.iterrows():
            mh_op = make_minhash(row_op["_ns"], num_perm)
            if mh_op is None:
                continue
            for key in lsh.query(mh_op):
                med_idx = int(key.split("_")[1])
                lsh_pairs.add((idx_op, med_idx))

        del lsh; gc.collect()

    elapsed = time.time() - t0
    n_pairs = len(lsh_pairs)
    rr = 1 - n_pairs / fullcross if fullcross > 0 else 0

    # Pairs completeness vs Strategy B
    pc_vs_B = len(lsh_pairs & set_B) / len(set_B) * 100 if len(set_B) > 0 else 0

    # Unique pairs not in B
    unique_to_lsh = len(lsh_pairs - set_B)

    print(f"  Pairs: {n_pairs:,}  RR: {rr*100:.6f}%  "
          f"PC vs B: {pc_vs_B:.1f}%  Unique: {unique_to_lsh:,}  "
          f"Time: {elapsed:.1f}s")

    sweep_results.append({
        "num_perm": num_perm,
        "threshold": threshold,
        "candidate_pairs": n_pairs,
        "reduction_ratio": rr,
        "pc_vs_strategy_B": round(pc_vs_B, 2),
        "unique_to_lsh": unique_to_lsh,
        "overlap_with_B": n_pairs - unique_to_lsh,
        "runtime_sec": round(elapsed, 1),
        "pairs_per_sec": round(n_pairs / elapsed, 0) if elapsed > 0 else 0
    })

sweep_df = pd.DataFrame(sweep_results)
print("\n" + "=" * 60)
print("SWEEP COMPLETE")
print("=" * 60)
print(sweep_df.to_string(index=False))



--- num_perm=128, threshold=0.5 ---
  Pairs: 100,196  RR: 99.998180%  PC vs B: 8.6%  Unique: 63,483  Time: 1695.8s

SWEEP COMPLETE
 num_perm  threshold  candidate_pairs  reduction_ratio  pc_vs_strategy_B  unique_to_lsh  overlap_with_B  runtime_sec  pairs_per_sec
      128        0.5           100196         0.999982              8.58          63483           36713       1695.8           59.0


## 4.5 Optimal Configuration Analysis

Select the configuration that maximizes pairs completeness while maintaining
>99.99% reduction ratio, and analyze its output in detail.

In [17]:
print("=" * 60)
print("4.5 OPTIMAL CONFIGURATION ANALYSIS")
print("=" * 60)

# Filter configs with RR > 99.99%
viable = sweep_df[sweep_df["reduction_ratio"] > 0.9999].copy()

if len(viable) == 0:
    print("WARNING: No configs meet RR > 99.99%. Relaxing to > 99.9%.")
    viable = sweep_df[sweep_df["reduction_ratio"] > 0.999].copy()

# Best = highest PC vs B among viable
best = viable.loc[viable["pc_vs_strategy_B"].idxmax()]
print(f"Best config: num_perm={int(best['num_perm'])}, threshold={best['threshold']}")
print(f"  Candidate pairs:    {int(best['candidate_pairs']):,}")
print(f"  Reduction ratio:    {best['reduction_ratio']*100:.6f}%")
print(f"  PC vs Strategy B:   {best['pc_vs_strategy_B']:.1f}%")
print(f"  Unique to LSH:      {int(best['unique_to_lsh']):,}")
print(f"  Runtime:            {best['runtime_sec']:.1f}s")

# Efficiency comparison
print(f"\n--- Efficiency vs Baselines ---")
print(f"Exact NPI:         {elapsed_npi:.2f}s → {len(npi_overlap):,} matches (tier-1 only)")
print(f"Strategy B:        {elapsed_B:.2f}s → {len(set_B):,} pairs (tier-2)")
print(f"Best LSH:          {best['runtime_sec']:.1f}s → {int(best['candidate_pairs']):,} pairs (tier-2)")


4.5 OPTIMAL CONFIGURATION ANALYSIS
Best config: num_perm=128, threshold=0.5
  Candidate pairs:    100,196
  Reduction ratio:    99.998180%
  PC vs Strategy B:   8.6%
  Unique to LSH:      63,483
  Runtime:            1695.8s

--- Efficiency vs Baselines ---
Exact NPI:         0.65s → 542,329 matches (tier-1 only)
Strategy B:        0.84s → 427,752 pairs (tier-2)
Best LSH:          1695.8s → 100,196 pairs (tier-2)


## 4.6 LSH + NPI Complementarity Analysis

The real value of LSH is finding matches that exact NPI **cannot** — tier-2 records
without NPIs. Here we quantify the additive value of LSH on top of NPI matching.

In [18]:
print("=" * 60)
print("4.6 LSH + NPI COMPLEMENTARITY")
print("=" * 60)

# Tier-1: exact NPI covers 933,615 OP records → matched to Medicare via NPI
# Tier-2: 12,813 OP records with NO NPI → must use fuzzy/LSH
# Unmatchable: 31,424 → excluded

# Of the tier-2 records, how many does each strategy cover?
# OP coverage = unique OP indices in candidate pairs

# Strategy B coverage
op_covered_B = len(set(p[0] for p in set_B))

# Best LSH coverage (we need to re-run at best config, or use sweep data)
# For now, use the pairs count and estimate
print(f"Tier-2 OP records total:  {len(op_tier2):,}")
print(f"Strategy B OP coverage:   {op_covered_B:,} ({op_covered_B/len(op_tier2)*100:.1f}%)")

print(f"\n--- Combined Pipeline Coverage ---")
n_total_op = len(op_tier1) + len(op_tier2) + len(op_unmatchable)
n_tier1_matched = len(npi_overlap)  # Approximate: unique NPIs overlapping
n_tier2_covered_B = op_covered_B

print(f"Total OP records:         {n_total_op:,}")
print(f"  Tier-1 NPI matched:     {n_tier1_matched:,} ({n_tier1_matched/n_total_op*100:.1f}%)")
print(f"  Tier-2 B-covered:       {n_tier2_covered_B:,} ({n_tier2_covered_B/n_total_op*100:.1f}%)")
print(f"  Unmatchable:            {len(op_unmatchable):,} ({len(op_unmatchable)/n_total_op*100:.1f}%)")
print(f"  Overall linkable:       {n_tier1_matched + n_tier2_covered_B:,} "
      f"({(n_tier1_matched + n_tier2_covered_B)/n_total_op*100:.1f}%)")


4.6 LSH + NPI COMPLEMENTARITY
Tier-2 OP records total:  4,683
Strategy B OP coverage:   4,574 (97.7%)

--- Combined Pipeline Coverage ---
Total OP records:         969,703
  Tier-1 NPI matched:     542,329 (55.9%)
  Tier-2 B-covered:       4,574 (0.5%)
  Unmatchable:            31,405 (3.2%)
  Overall linkable:       546,903 (56.4%)


## 4.7 Runtime Scaling Analysis

Model how LSH runtime scales with dataset size and signature length.

In [19]:
print("=" * 60)
print("4.7 RUNTIME SCALING ANALYSIS")
print("=" * 60)

# Group by num_perm to see scaling
for np_val in NUM_PERMS:
    subset = sweep_df[sweep_df["num_perm"] == np_val]
    avg_time = subset["runtime_sec"].mean()
    print(f"num_perm={np_val:>3}: avg runtime {avg_time:.1f}s across {len(subset)} thresholds")

# Theoretical scaling
print(f"\n--- Theoretical Scaling ---")
n_op = len(op_tier2)
n_med = len(medclean)
print(f"Current scale: {n_op:,} OP × {n_med:,} Med = {n_op * n_med:,.0f} potential pairs")

# If we scaled to 14.7M OP records (full row-level):
n_op_full = 14_700_000
scale_factor = n_op_full / n_op
print(f"Full-scale OP: {n_op_full:,} (scale factor: {scale_factor:.0f}x)")
print(f"Projected LSH runtime at full scale:")
for _, row in sweep_df.iterrows():
    projected = row["runtime_sec"] * scale_factor
    print(f"  perm={int(row['num_perm']):>3}, thresh={row['threshold']:.1f}: "
          f"{projected/60:.0f} min ({projected/3600:.1f} hr)")

print(f"\nFor comparison:")
print(f"  Full cross-join at full scale: {n_op_full * n_med:,.0f} pairs (infeasible)")
print(f"  Strategy B scales linearly with block sizes (minutes)")
print(f"  LSH scales ~O(n) per state with constant overhead for hashing")


4.7 RUNTIME SCALING ANALYSIS
num_perm=128: avg runtime 1695.8s across 1 thresholds

--- Theoretical Scaling ---
Current scale: 4,683 OP × 1,175,281 Med = 5,503,840,923 potential pairs
Full-scale OP: 14,700,000 (scale factor: 3139x)
Projected LSH runtime at full scale:
  perm=128, thresh=0.5: 88719 min (1478.6 hr)

For comparison:
  Full cross-join at full scale: 17,276,630,700,000 pairs (infeasible)
  Strategy B scales linearly with block sizes (minutes)
  LSH scales ~O(n) per state with constant overhead for hashing


## 4.8 Export Results

In [20]:
print("=" * 60)
print("4.8 EXPORT")
print("=" * 60)

# Save sweep results
sweep_df.to_csv(os.path.join(OUTPUTDIR, "lsh_benchmark_results.csv"), index=False)
print(f"✓ lsh_benchmark_results.csv ({len(sweep_df)} rows)")

# Save baselines comparison
baselines = pd.DataFrame([
    {"method": "Exact NPI", "scope": "tier-1", "matches_or_pairs": len(npi_overlap),
     "runtime_sec": round(elapsed_npi, 2), "reduction_ratio": None,
     "note": "Gold standard for tier-1; cannot handle tier-2"},
    {"method": "Strategy B (State+Soundex)", "scope": "tier-2", 
     "matches_or_pairs": len(set_B), "runtime_sec": round(elapsed_B, 2),
     "reduction_ratio": round(1 - len(set_B)/fullcross, 8),
     "note": "Best traditional blocking from Phase 3"},
    {"method": f"LSH (perm={int(best['num_perm'])}, thresh={best['threshold']})", 
     "scope": "tier-2",
     "matches_or_pairs": int(best["candidate_pairs"]),
     "runtime_sec": best["runtime_sec"],
     "reduction_ratio": round(best["reduction_ratio"], 8),
     "note": "Best LSH config from sweep"},
])
baselines.to_csv(os.path.join(OUTPUTDIR, "lsh_vs_baselines.csv"), index=False)
print(f"✓ lsh_vs_baselines.csv ({len(baselines)} rows)")

# Cleanup
op_tier2.drop(columns="_ns", inplace=True, errors="ignore")
medclean.drop(columns="_ns", inplace=True, errors="ignore")

print(f"\nAll artifacts saved to {OUTPUTDIR}")
for f in sorted(os.listdir(OUTPUTDIR)):
    sz = os.path.getsize(os.path.join(OUTPUTDIR, f)) / 1024
    print(f"  {f:40s} {sz:8.1f} KB")

print("\n" + "=" * 60)
print("GAP 4 COMPLETE")
print("=" * 60)


4.8 EXPORT
✓ lsh_benchmark_results.csv (1 rows)
✓ lsh_vs_baselines.csv (3 rows)

All artifacts saved to ../artifacts/phase6_lsh_benchmark
  lsh_benchmark_results.csv                     0.2 KB
  lsh_vs_baselines.csv                          0.3 KB

GAP 4 COMPLETE
